[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Pyrcel/condensation_playground.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/claresinger/PySDM-examples/blob/ese134/PySDM_examples/Pyrcel/condensation_playground.ipynb)

## Cloud Microphysics: Part 1
- Activation of aerosol particles into cloud droplets
- Exploring how size/composition affect condensational growth

Based on Example Figure from Pyrcel code documentation https://pyrcel.readthedocs.io/en/latest/examples/basic_run.html

## Droplet activation 
(for more info read Ch. 6 of Lohmann's _An Introduction to Clouds_)
#### Köhler curve:
- Curvature effect (Kelvin equation), describes the increase in vapor pressure over a curved surface
compared to a flat surface and how this depends on the surface tension and radius of the droplet.

$e_s(r) = e_s(\infty) \exp \left( \frac{2 \sigma}{r \rho R_v T} \right)$

- Solute effect (Raoult's law), describes the reduction of vapor pressure over a flat surface due
to the presence of soluble material, aka aerosol.

$\kappa = \frac{1}{a_w} = \frac{e_s(\infty)}{e_*(\infty)} = \frac{n_s + n_w}{n_w}$

The hygroscopicity (inverse of water activity) is defined as the ratio of the total number of 
moles of solute plus water to the number of moles of water.

- Putting it together, the Köhler curve, or $\kappa$-Köhler curve, describes the hygroscopic 
growth of particles, and the maximum of this curve, describes the point of activation from
an aerosol into a cloud droplet.

$S(r) = \frac{e_*(r)}{e_s(\infty)} \approx 1 + \frac{a}{r} - \frac{b}{r^3}$

<img src="./kohler.png" width="500">
Fig 6.11 from Lohamann. You can see a characteristic Köhler curve with the critical radius ($r_{act}$) and supersaturation ($S_{act}$) which separate the stable (aerosol or "solution droplet") and unstable (cloud droplet) regimes labeled.

#### Other considerations:
- Surface tension: The surface tension $\sigma$ in the Kelvin equation is usually assumed as
constant $\sigma = \sigma_w = 72$ mN, but complex chemistry of the aerosol can sometimes actually
modify the effective surface tension of the growing cloud droplet.


## PySDM parcel model widget

In this homework assignment, and with this `PySDM` example notebook, you have the chance to explore how particle size, number concentration, and chemical composition, influence the bulk properties of a cloud by using a parcel model.

A parcel model takes a parcel of air and lifts it adiabatically, greatly simplifying the dynamics taking place in a real cloud, but resolving the microphysical processes we are interested in exploring here.

You can play around with the widget at the bottom to change the initial aerosol properties, while keeping the dynamics fixed (i.e. updraft velocity `w = 1 * si.m / si.s` or temperature `T0 = 274 * si.K`). 

In [1]:
# import PySDM-examples library on google colab
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install "atmos-cloud-sim-uj-utils"
    from atmos_cloud_sim_uj_utils import pip_install_on_colab
    pip_install_on_colab(
        'PySDM-examples@git+https://github.com/claresinger/PySDM-examples@ese134#egg=PySDM-examples'
        )

In [6]:
# import functions for creating interactive widget
import numpy as np
from numpy import errstate
from IPython.display import display
from PySDM_examples.utils import widgets

# import PySDM tools for initializing and running a cloud parcel model
from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement, AmbientTemperature, AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume, ParticleVolumeVersusRadiusLogarithmSpectrum
)

# import tools for running and plotting this example
from PySDM_examples.Pyrcel import Settings, Simulation, ProfilePlotter

In [3]:
# create progress bar for widget
progbar = widgets.IntProgress(min=0, max=100, description='%')

In [4]:
# create initial aerosol distribution
# run cloud parcel model
# save and plot results

def demo(*, freezer, k2, N2, r2):
    with freezer:
        with errstate(all='raise'):
            settings = Settings(
                dz = 1 * si.m,
                n_sd_per_mode = (10, 10),
                aerosol_modes_by_kappa = {
                    .54: Lognormal(
                        norm_factor=850 / si.cm ** 3,
                        m_mode=15 * si.nm,
                        s_geom=1.6
                    ),
                    k2: Lognormal(
                        norm_factor=N2 / si.cm ** 3,
                        m_mode=r2 * si.nm,
                        s_geom=1.2
                    )
                },
                vertical_velocity = 1.0 * si.m / si.s,
                initial_pressure = 775 * si.mbar,
                initial_temperature = 274 * si.K,
                initial_relative_humidity = 0.98,
                displacement = 250 * si.m,
                formulae = Formulae(constants={'MAC': .3})
            )
            dry_radius_bin_edges = np.logspace(
                np.log10(1e-3 * si.um),
                np.log10(5e0 * si.um),
                33, endpoint=False
            )
            simulation = Simulation(
                settings,
                products=(
                ParcelDisplacement(
                    name='z'),
                AmbientRelativeHumidity(
                    name='S_max', unit='%', var='RH'),
                AmbientTemperature(
                    name='T'),
                ParticleSizeSpectrumPerVolume(
                    name='dry:dN/dR', radius_bins_edges=dry_radius_bin_edges, dry=True),
                ParticleVolumeVersusRadiusLogarithmSpectrum(
                    name='dry:dV/dlnR', radius_bins_edges=dry_radius_bin_edges, dry=True),
                ),
                scipy_solver=False,
            )
            output = simulation.run((widgets.ProgbarUpdater(progbar, settings.output_steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = ProfilePlotter(settings)
            plotter.plot(output)
            plotter.show()
            
            

In [5]:
# create widget
# use to explore how the hygroscopicity, number concentration, and mean radius
# of the initial aerosol distribution affect the bulk parcel properties
# like maximum supersaturation and temperature profile

k2 = widgets.FloatSlider(value=1.2, min=0.2, max=1.4, step=0.1, description='κ2', continuous_update=False, readout_format='.1f')
N2 = widgets.IntSlider(value=10, min=5, max=50, step=5, description='N2', continuous_update=False)
r2 = widgets.IntSlider(value=850, min=200, max=1000, step=50, description='r2', continuous_update=False)
sliders = widgets.HBox([k2, N2, r2])
freezer = widgets.Freezer([k2, N2, r2])
self = widgets.interactive_output(demo, {'freezer': freezer, 'k2': k2, 'N2': N2, 'r2': r2})

display(sliders, progbar, self)

IntProgress(value=100, description='%')

Output()

## Questions

1. Extremes: Which combination of (changeable) parameters leads to the largest maximum supersaturation? Which to the smallest? Why?

2. Sensitivity: Is the cloud more sensitive to changes in aerosol size, number, or composition? Explain how you are measuring this.

3. Albedo: The albedo of a cloud is very dependent on the size of the individual droplets it is composed of. 
We can express the cloud albedo ($\alpha$) in terms of the cloud optical thickness ($\tau$) and a dimensionless asymmetry parameter ($g$)
that describes the relative amount of radiation that is forward-scattered vs. backscattered. 

$\alpha = \frac{(1-g) \tau}{2 + (1-g)\tau}$

Cloud droplets (order 1-10$\mu$m) tend to be strongly forward-scattering with an asymmetry parameter around $g=0.85$.
The cloud optical thickness can be written in terms of the liquid water path through the cloud (LWP) and effective radius of the droplets ($r_e$).

$\tau = \frac{3 LWP}{2 \rho_w r_e}$

a) With the assumption of fixed LWP, write down an expression for the cloud albedo as a function of the droplet effective radius.
This dependence of cloud albedo on droplet size is known as the "Twomey effect."

b) Describe how the albedo would change given changes to the initial aerosol size distribution. 